In [ ]:
# File: fitting.ipynb
# Author: Ryoichi Ando (ryoichi.ando@zozo.com)
# License: Apache v2.0

In [ ]:
import os
import random

from frontend import App

app = App.create("fitting")

codim_ipc_root = "/tmp/Codim-IPC"
checkout_list = [
    "Projects/FEMShell/input/dress_knife",
    "Projects/FEMShell/input/Rumba_Dancing",
]
app.extra.sparse_clone(
    "https://github.com/ipc-sim/Codim-IPC", codim_ipc_root, checkout_list
)

stage_path = os.path.join(codim_ipc_root, checkout_list[0], "stage.obj")
body_path = os.path.join(codim_ipc_root, checkout_list[1], "shell0.obj")

V, F, S = app.extra.load_CIPC_stitch_mesh(stage_path)
app.asset.add.tri("dress", V, F)
app.asset.add.stitch("glue", S)

V, F = app.mesh.load_tri(body_path)
app.asset.add.tri("body", V, F)

scene = app.scene.create()
dress = scene.add("dress")
dress.param.set("friction", 0.0).set("strain-limit", 0.1)
dress.stitch("glue").rotate(-90, "x")

jitter = 0.01 * random.random()
body = scene.add("body").at(0, -0.78 + jitter, 0)
body.param.set("friction", 0.0).set("model", "arap")
pin = body.pin()
body_dir = os.path.join(codim_ipc_root, checkout_list[1])
frame = 1
while True:
    path = os.path.join(body_dir, f"shell{frame}.obj")
    if os.path.exists(path):
        V, _ = app.mesh.load_tri(path)
        pin.move_to(V, 0.5 + 0.025 * frame)
        frame += 1
    else:
        break

scene = scene.build().report()
scene.preview(options={"pin": False})

In [ ]:
session = app.session.create(scene)
param = session.param.set("fitting").set("dt", 1e-3).set("frames", 240)
param.dyn("fitting").time(0.15).hold().change(False)
param.dyn("dt").time(0.15).hold().change(0.01)
session = session.build()

In [ ]:
session.start().preview()
session.stream()

In [ ]:
session.animate()

In [ ]:
session.export.animation()

In [ ]:
# this is for CI
assert session.finished()